In [1]:
import pandas as pd
from datetime import datetime
import gspread as gs
import numpy as np

In [2]:
#ejecutar para crear un nuevo archivo en google drive la primera vez

# sh = gc.create('documentos_procesado')

In [3]:
# Ruta del archivo .xlsx
file_path = "revision_documentos.xlsx"
file_path_destino = "revision_documentos_procesado_" + datetime.now().strftime("%Y%m%d%H%M") + ".xlsx"
credentials_filename='credenciales\client_secret_290615798724-sg1nk4latali6a8akhe169r66gg7hmd8.apps.googleusercontent.com.json'
# Lee el archivo Excel
df = pd.read_excel(file_path)


In [4]:
# Para el primer analisis, se eliminan las columnas que no son relevantes
# se eliminan las columnas: DOCUMENTOVIGENCIA, FECHA_INICIO_DOCUMENTO, FECHA_FIN_DOCUMENTO
# DOCO_EMPR_RUT, TIPO, 'Hora extendida', 'Fecha y hora', 'Fecha.1', 'Gestionado','Tiempo desde ingreso', 'Tiempo para cumplir en plazo', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43' 'Unnamed: 29','Unnamed: 30', 'Fecha', 'Hora', 'AM/PM'
columnas_no_tenidas_en_cuenta = [
    'DOCUMENTOVIGENCIA', 'FECHA_INICIO_DOCUMENTO', 'FECHA_FIN_DOCUMENTO', 'DOCO_EMPR_RUT',
    'TIPO', 'Hora extendida', 'Fecha y hora', 'Fecha.1', 'Gestionado',
     'Tiempo desde ingreso', 'Tiempo para cumplir en plazo', 'Unnamed: 40',
     'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 29','Unnamed: 30',
     'Fecha', 'Hora', 'AM/PM']

df.drop(columnas_no_tenidas_en_cuenta,
        axis=1, inplace=True)

#verificamos cuantos usuarioregistroid son nulos
print(df['USUARIOREGISTROID'].isnull().sum())

#en la columna USUARIOREGISTROID, reemplazamos los valores nulos por -1
df['USUARIOREGISTROID'].fillna(-1, inplace=True)

299


In [5]:
# Creamos la columna tipo, en base a la columna estado. 
# Si el estado es "Enviado", el tipo es de "Entrada"
# Si el estado es "Rechazado" o "Validado", el tipo es de "Salida"

df["TIPO"] = df["ESTADO"].apply(lambda x: "Entrada" if x == "Enviado" else "Salida")

In [6]:
# Convertimos la columna FECHA_REGISTRO_HH, en formato mm/dd/yyyy hh:mm:ss PM/AM a datetime
df["FECHA_REGISTRO_HH"] = pd.to_datetime(df["FECHA_REGISTRO_HH"], format="%m/%d/%Y %I:%M:%S %p")

In [7]:
#creamos dos dataframes, uno con los documentos de entrada y otro con los de salida
df_entrada = df[df["TIPO"] == "Entrada"]
df_salida = df[df["TIPO"] == "Salida"]

In [8]:
print(df_entrada.shape)
print(df_salida.shape)

columnas_eliminadas_salida = ['FECHA_REGISTRO', 'ORIGENDOCID', 'ORIGEN',
       'DOCO_TIDO_ID', 'ID_ESTADO',
       'DOCUMENTOSITUACIONID', 'DOCUMENTOSITUCION', 'USUARIOREGISTRO',
       'USUARIOREGISTROCUENTA', 'USUARIOREGISTRONOMBRE',
       'DOCO_TIPO_VALIDACION', 'DOCO_CONT_ID', 'FECHA_INGRESO_DOCUMENTO'
       , 'DOCUMENTOESPECIALIDADID', 'DOCUMENTOESPECIALIDAD',
       'DOCUMENTOOCUPACIONID', 'DOCUMENTOOCUPACION',
       'DOCUMENTOCERTOCUPACIONID', 'DOCUMENTOCERTOCUPACION', 'TIPO']

(28869, 25)
(26718, 25)


In [9]:
# del dataframe de salidas, nos interesa conservar las columnas
# FECHA_REGISTRO_HH, ESTADO, USUARIOREGISTROID, pero renombramos a 
# FECHA_SALIDA, ESTADO_SALIDA, USUARIOID_SALIDA  para luego poder hacer el merge
df_salida = df_salida.rename(columns={"FECHA_REGISTRO_HH": "FECHA_SALIDA", "ESTADO": "ESTADO_SALIDA", "USUARIOREGISTROID": "USUARIOID_SALIDA"})
df_salida.drop(
       columnas_eliminadas_salida
       , axis=1, inplace=True)


In [10]:
# hacemos el merge, en base al doco_id, lo hacemos de tipo left para conservar los documentos de entrada
df_merged = pd.merge(df_entrada, df_salida, on="DOCO_ID", how="left")


El código de abajo es para ver si hay documentos que tienen salida pero no entrada, que en teoría no debería ocurrir
Una de las causas puede ser que la entrada este en otro documento

In [11]:
# Realizamos el merge de tipo right para conservar todos los documentos de salida
df_right_merged = pd.merge(df_entrada, df_salida, on="DOCO_ID", how="right")

# Filtramos los registros donde ID_ESTADO de entrada sea NaN
df_salida_sin_entrada = df_right_merged[df_right_merged['ID_ESTADO'].isna()]

# Mostramos los documentos de salida que no tienen un documento de entrada correspondiente
# print(df_salida_sin_entrada)

In [12]:
#Agregamos una columna con la diferencia de tiempo entre entrada y salida de los documentos en horas 
# en caso de no haber fecha salida, la columna se rrrellena con NaN
# en caso de no haber fecha de registro, la columna se rellena con NaN

df_merged["DIFERENCIA_HORAS"] = df_merged.apply(
    lambda row: (row["FECHA_SALIDA"] - row["FECHA_REGISTRO_HH"]).total_seconds() / 3600
    if pd.notnull(row["FECHA_SALIDA"]) and pd.notnull(row["FECHA_REGISTRO_HH"]) else np.nan,
    axis=1
)

In [14]:
# Definimos el umbral de 48 horas
umbral = 48

# Agregamos un campo de desviación respecto al umbral
df_merged["DESVIACION"] = df_merged["DIFERENCIA_HORAS"] - umbral

# Determinamos el estado, teniendo en cuenta la desviación y los documentos
# sin salida

df_merged['estado'] = df_merged.apply(
    lambda row: 'Sin gestionar' if pd.isna(row['FECHA_SALIDA'])
    else 'Sin entrada' if pd.isna(row['FECHA_REGISTRO_HH'])
    else 'Fuera de plazo' if row['DESVIACION'] > 0
    else 'En plazo' if row['DESVIACION'] < 0
    else 'Igual', 
    axis=1
)
    

In [15]:
# Obtener el mes actual y los tres meses anteriores
hoy = datetime.now()
mes_actual = hoy.month
año_actual = hoy.year

# Crear una lista de los tres meses anteriores y el mes actual
meses_validos = [(mes_actual - i - 1) % 12 + 1 for i in range(4)]
año_valido = año_actual if mes_actual in meses_validos else año_actual - 1

# Función para obtener la etiqueta del mes
def etiqueta_mes(fecha):
    mes = fecha.month
    año = fecha.year
    
    if mes == mes_actual and año == año_actual:
        return f"{hoy.strftime('%B')} {año_actual}"
    
    if mes in meses_validos:
        return f"{fecha.strftime('%B')} {año}"
    
    return 'Fuera de rango'

# Aplicar la función al DataFrame
df_merged['etiqueta_mes'] = df_merged['FECHA_REGISTRO_HH'].apply(etiqueta_mes)

In [16]:
# Exportamos del DataFrame a un archivo Excel
#df_merged.to_excel(file_path_destino, index=False)

# TODO sacar comentario

In [ ]:
# creamos un filtro para los documentos en bse a fehca_salida 
fecha_inicial = datetime(2024, 8, 1)
fecha_final = datetime(2024, 8, 10)

filtro_fecha_salida = (df_merged['FECHA_SALIDA'] >= fecha_inicial) & (df_merged['FECHA_SALIDA'] <= fecha_final)

In [19]:
#Preparamos la exprotación a sheets
#  es necesario convertir los timestamps a string, en formato yyyy-mm-dd hh:mm:ss
# convertimos los timestamps a string, teniendo en cuenta que pueden haber NaTs

df_merged = df_merged.applymap(
    lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
    if isinstance(x, (pd.Timestamp, datetime)) and not pd.isnull(x) else x
)

# reemplazamos NaNs con espccios vacíos
df_merged= df_merged.applymap(lambda x: '' if pd.isnull(x) else x)



In [ ]:
gc = gs.oauth(credentials_filename)

In [21]:
# exportamos a google sheets

sh = gc.open('documentos_procesado')
worksheet = sh.sheet1
worksheet.update([df_merged.columns.values.tolist()] + df_merged.values.tolist())

{'spreadsheetId': '1xXVP33Zg4bzxH-xUvKZRxeTvhsRdlDDfzIXqc2ttHus',
 'updatedRange': "'Hoja 1'!A1:AG28960",
 'updatedRows': 28960,
 'updatedColumns': 33,
 'updatedCells': 955680}